In [2]:
import time
import os
import glob
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Load park codes from external JSON file
json_file = "np_parks.json"
with open(json_file, "r") as file:
    np_parks = json.load(file)

# Define directories
download_dir = r"C:\Users\dhair\Downloads\STA220"
output_file = os.path.join(download_dir, "all_parks_species.csv")

# Ensure the directory exists
os.makedirs(download_dir, exist_ok=True)

# Set up Chrome options with the download directory
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_argument("--start-maximized")

# Initialize the driver
driver = webdriver.Chrome(options=chrome_options)

def download_park_species(park_code, wait_time=10):
    print(f"Processing park: {park_code}")
    driver.get(f"https://irma.nps.gov/NPSpecies/Search/SpeciesList/{park_code}")
    time.sleep(10)
    
    try:
        # First select the "Full list with details" radio button
        try:
            radio_button = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, "//label[contains(text(), 'Full list with details')]/preceding-sibling::input[@type='button'] | //label[contains(text(), 'Full list with details')]"))
            )
            radio_button.click()
            print(f"Selected 'Full list with details' for {park_code}")
            time.sleep(2)
        except Exception as e:
            print(f"Could not select 'Full list with details' radio button: {e}")
            # Try JavaScript click as a fallback
            try:
                driver.execute_script("""
                    var labels = document.querySelectorAll('label.x-form-cb-label');
                    for (var i = 0; i < labels.length; i++) {
                        if (labels[i].textContent.includes('Full list with details')) {
                            labels[i].click();
                            return true;
                        }
                    }
                """)
                time.sleep(2)
            except Exception as js_error:
                print(f"JavaScript click failed: {js_error}")
        
        # Click the Search button
        try:
            search_button = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.ID, "button-1021"))
            )
            search_button.click()
            time.sleep(5)
        except Exception as e:
            print(f"Could not click Search button by ID: {e}")
            try:
                search_buttons = driver.find_elements(By.XPATH, "//span[contains(text(), 'Search')]/.. | //a[contains(text(), 'Search')] | //button[contains(text(), 'Search')]")
                if search_buttons:
                    for button in search_buttons:
                        try:
                            button.click()
                            time.sleep(10)
                            break
                        except:
                            continue
            except Exception as e:
                print(f"All search button attempts failed: {e}")
        
        # Click Download button
        try:
            download_button = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.ID, "button-1142"))
            )
            download_button.click()
            print(f"Clicked Download button for {park_code}")
            time.sleep(wait_time)
            return True
        except Exception as e:
            try:
                download_links = driver.find_elements(By.XPATH, "//span[contains(text(), 'Download')]/.. | //a[contains(text(), 'Download')] | //button[contains(text(), 'Download')]")
                if download_links:
                    for link in download_links:
                        try:
                            link.click()
                            print("Clicked Download button")
                            time.sleep(wait_time)
                            return True
                        except:
                            continue
                download_clicked = driver.execute_script("""
                    var spans = document.querySelectorAll('span.x-btn-inner');
                    for (var i = 0; i < spans.length; i++) {
                        if (spans[i].textContent.includes('Download')) {
                            spans[i].click();
                            return true;
                        }
                    }
                    return false;
                """)
                if download_clicked:
                    time.sleep(wait_time)
                    return True
                else:
                    print(f"Could not find any Download button for {park_code}")
                    return False
            except Exception as e:
                print(f"All download button attempts failed: {e}")
                return False
    except Exception as e:
        print(f"Error processing park {park_code}: {e}")
        return False

try:
    for park_name, park_code in np_parks.items():
        success = download_park_species(park_code)
        if not success:
            print(f"Failed to download data for {park_name} ({park_code})")
        time.sleep(5)
    
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    print("Closing browser...")
    driver.quit()

Processing park: CHIS
Selected 'Full list with details' for CHIS
Clicked Download button for CHIS
Processing park: DEVA
Selected 'Full list with details' for DEVA
Clicked Download button for DEVA
Processing park: JOTR
Selected 'Full list with details' for JOTR
Clicked Download button for JOTR
Processing park: KICA
Selected 'Full list with details' for KICA
Clicked Download button for KICA
Processing park: LAVO
Selected 'Full list with details' for LAVO
Clicked Download button for LAVO
Processing park: PINN
Selected 'Full list with details' for PINN
Clicked Download button for PINN
Processing park: REDW
Selected 'Full list with details' for REDW
Clicked Download button for REDW
Processing park: SEQU
Selected 'Full list with details' for SEQU
Clicked Download button for SEQU
Processing park: YOSE
Selected 'Full list with details' for YOSE
Clicked Download button for YOSE
Closing browser...
